In [1]:
"""Helper Functions for the larger project"""
from random import choice
from random import randint 
from math import log10

#File reader to read in fasta files 
def fasta_reader(file):
    return [line.strip("\n").split(",") for line in open(file)]

#generate a random character given weights 
def select_char(probabilities:dict, precision: int = 1000):
    s = ""
    for i in probabilities.keys():
        s+=i*int(probabilities[i]*1000)
    return choice(s) 
#Viterbi
def rough_viterbi(states, transition_matrix, seq, logodd=0):
    possible_state = {} #Use this to store results 
    if logodd == 0:
        for i in states:
            prob = 1
            prev_char = ""
            for s in seq:
                if prev_char != "":
                    pass
                else:
                    prob*=transition_matrix[i][s][s]
            possible_state[i] = prob
    else:
        for i in states:
            prob = 1
            prev_char = ""
            for s in seq:
                if prev_char != "":
                    pass
                else:
                    prob*=log10(transition_matrix[i][s][s])
            possible_state[i] = prob
        
    highest = [0.0, ""]
    for i in possible_state.keys():
        if possible_state[i] > highest[0]:
            highest = [possible_state[i], i]
    return highest

def log(var, logodd):
    if var == 1 or logodd == 0:
        return var
    elif logodd != 0 and var !=1:
        return log10(var)
def smooth_viterbi(states, transition_matrix, start_matrix, seq, logodd=0, shift = .1, verbose=0):
    state_path = []
    prob = 1
    prev_char = ""
    prev_predicted_state = ""
    for s in seq:
        if prev_char != "":
            possible_states = {i:0 for i in states}
            for i in transition_matrix.keys():
                if i == prev_predicted_state:
                    possible_states[i] = log((1-(shift)*len(states))*transition_matrix[i][prev_char][s], logodd)
                else:
                    possible_states[i] = log(shift * transition_matrix[i][prev_char][s], logodd)
            trans_prob = -1000
            state_prob = ""
            for i in possible_states.keys():
                if possible_states[i] > trans_prob:
                    trans_prob = possible_states[i]
                    state_prob = i
            prev_char = s
            prev_predicted_state = state_prob
            prob*=trans_prob
            state_path.append(prev_predicted_state)                                         
        else:
            start_probs = [(log(start_matrix[i][s], logodd), i) for i in start_matrix.keys()]
            start_prob = -1000
            for i in start_probs:
                if i[0] > start_prob:
                    start_prob = i[0]
                    prev_predicted_state = i[1]
            prob*=start_prob
            prev_char=s
            state_path.append(prev_predicted_state)
    results = (prob, max(set(state_path), key=state_path.count))
    if verbose != 0:
        print(state_path)
        return (state_path, prob, results)
    else:
        return results

"""The class which does it all"""    
class HMM:
    def __init__(self):
        self.states = []
        self.emissions = []
        self.data_table = {}
        self.transition_matrix = {}
        self.start_matrix ={}
    def load_data(self, file, state_slice=0, seq_slice=2):
        for data_point in fasta_reader(file):
            state = data_point[state_slice]
            seq = data_point[seq_slice].strip(" ")
            if state not in self.states:
                self.states.append(state)
            for i in seq:
                if i not in self.emissions and i != " ":
                    self.emissions.append(i)
            self.data_table[seq] = state 

    def train_model(self):
        for state in self.states:
            t = {}
            for j in self.emissions:
                t[j] = {z:1 for z in self.emissions}
            self.transition_matrix[state] = t
            self.start_matrix[state] = {i:1  for i in self.emissions}
        for seq in self.data_table.keys():
            state = self.data_table[seq]
            prev_char = 0
            # currently assumes start chars have no special properties 
            for char in seq: 
                if prev_char == 0:
                    #self.transition_matrix[state][char][char] = 1 + self.transition_matrix[state][char][char] 
                    self.start_matrix[state][char] = 1 + self.start_matrix[state][char]
                    pass
                else:
                    self.transition_matrix[state][prev_char][char] = 1 + self.transition_matrix[state][prev_char][char]
                prev_char = char
        for state in self.transition_matrix.keys():
            for em in self.transition_matrix[state].keys():
                total_count = 0
                for em2 in self.transition_matrix[state][em].keys():
                    total_count += self.transition_matrix[state][em][em2]
                for em2 in self.transition_matrix[state][em].keys():
                    self.transition_matrix[state][em][em2] = self.transition_matrix[state][em][em2]/total_count
        for state in self.start_matrix.keys():
            total_count = 1
            for em in self.start_matrix[state]:
                total_count+= self.start_matrix[state][em]
            for em in self.start_matrix[state]:
                self.start_matrix[state][em] = self.start_matrix[state][em]/total_count
            

    def generate_seq(self, state, seq_length=60):
        s = ""
        prev_char = 0
        for i in range(seq_length):
            if prev_char != 0:
                t = select_char(self.transition_matrix[state][prev_char])
                s+=t
                prev_char = t
            else:
                t = choice(self.emissions)
                s+=select_char(self.transition_matrix[state][t])
                prev_char = t
        print(s)
        return s
    
    def generate_totally_random_seq(self, seq_length=60):
        state = choice(self.states)
        s = ""
        prev_char = 0
        for i in range(seq_length):
            if prev_char != 0:
                t = select_char(self.transition_matrix[state][prev_char])
                s+=t
                prev_char = t
            else:
                t = choice(self.emissions)
                s+=select_char(self.start_matrix[state])
                prev_char = t
        print(s)
        return s       
    
    def viterbi_score(self, seq, rs="r"):
        if rs == "r":
            return rough_viterbi(seq=seq, states=self.states, transition_matrix=self.transition_matrix)
        elif rs == "s":
            return smooth_viterbi(seq=seq, states=self.states, transition_matrix=self.transition_matrix, start_matrix=self.start_matrix, verbose=1)
        else:
            print("Impromper rs parameter")
    
    
    def how_kosher_am_i(self, alg=rough_viterbi):
        results = {i:{j:0 for j in self.states} for i in self.states}  
        for i in self.data_table.keys():
            if alg == rough_viterbi:
                score = alg(seq=i, states=self.states, transition_matrix=self.transition_matrix, logodd=0)
            elif alg == smooth_viterbi:
                score = alg(seq=i, states=self.states, transition_matrix=self.transition_matrix, logodd=1, start_matrix=self.start_matrix)
            results[self.data_table[i]][score[1]] = 1 +results[self.data_table[i]][score[1]]
        return results
            
            
                    
x = HMM()
x.load_data("Projects/splice.data.txt")  
x.train_model()
print(x.viterbi_score("GATCCCCCTGACCCAGCACCCCCTCCGCAGGTGCCGTGCCCCTCATCCAGTCTCGGATTG", rs="s"))
print("Smooth")
print(x.how_kosher_am_i(alg=smooth_viterbi))
print("vs. Rough")
print(x.how_kosher_am_i())
for i in range(10):
    s = x.generate_totally_random_seq()
    print(x.viterbi_score(s, rs="s"))
    


['EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI']
(['EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI', 'EI'], 1.77144385524311e-44, (1.77144385524311e-44, 'EI'))
Smooth
{'EI': {'EI': 207, 'IE': 320, 'N': 152}, 'IE': {'EI': 156, 'IE': 385, 'N': 131}, 'N': {'EI': 460, 'IE': 774, 'N': 420}}
vs. Rough
{'EI': {'EI': 327, 'IE': 15, 'N': 337}, 'IE': {'EI': 113, 'IE': 118, 'N': 441}, 'N': {'